Let's try a super-simple decision tree + pruning model, to see to what extent the data is non-linear. Our RMSE for one of the folds was 1881, for comparison.

In [10]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
# from DecisionTreeRegressor import DecisionTreeRegressor
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder


In [2]:
df = pd.read_hdf('../data/data_till_june_2016_w_descs_and_census.h5')
df = df.dropna(subset=['CLOSED_DT'])
df.shape

(718936, 155)

The performance btwn CV and test sets will prolly vary widely, bc the method is very sensitive to the data itself.

## Splitting the data

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('COMPLETION_TIME', axis=1), 
    df.COMPLETION_TIME, 
    test_size=0.2, 
    random_state=300
)

In [4]:
ss = ShuffleSplit(n_splits=5, test_size=0.2, random_state=300)

## Making the regressors

In [5]:
label_encoder = LabelEncoder()
label_encoder.fit(df['TYPE']);

LabelEncoder()

In [11]:
pipe1 = make_pipeline(DecisionTreeRegressor(max_depth=2))
pipe2 = make_pipeline(DecisionTreeRegressor(max_depth=5))

In [16]:
pipe = pipe2

In [17]:
training_scores = []
test_scores = []

for CV_train_index, CV_test_index in ss.split(y_train):
    X_CV_train = label_encoder.transform(X_train.iloc[CV_train_index]['TYPE'])[:, pd.np.newaxis]
    X_CV_test = label_encoder.transform(X_train.iloc[CV_test_index]['TYPE'])[:, pd.np.newaxis]
    y_CV_train = y_train.iloc[CV_train_index]
    y_CV_test = y_train.iloc[CV_test_index]
    
    pipe.fit(X_CV_train, y_CV_train)
    
    training_score = pipe.score(X_CV_train, y_CV_train)
    test_score = pipe.score(X_CV_test, y_CV_test)
    
    training_scores += [training_score]
    test_scores += [test_score]
    print "Training: {}. Test: {}".format(training_score, test_score)

Training: 0.217100930488. Test: 0.212150795535
Training: 0.220519238855. Test: 0.217272263857
Training: 0.216708247758. Test: 0.213712277052
Training: 0.217741260735. Test: 0.209185370479
Training: 0.216736523547. Test: 0.213529304421


I'm predicting quite different CV scores for each fold, and perhaps better performance than Linear Regression's $R^2$ of 0.24.

Funny--the error scores are all pretty similar. Not exactly sure why. Would need to investigate and think abou that more. Also, `max_depth=2` results in $R^2$ values around 11%. `max_depth=5` results in values around 21%.

I should think about what would be good intuitive values.